In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import math
from mplsoccer import PyPizza, add_image, FontManager
from highlight_text import fig_text
import requests
pd.options.mode.chained_assignment = None  # Disable the warning
pd.set_option('display.max_columns', None)

In [2]:
league_id = 48
season_id = 20721
league = 'championship'
no_games = 46

In [3]:
params ={
    'id':f'{league_id}',
    'season':f'{season_id}',
    'type':'teams',
    'stat':'goals_team_match',
    'slug':f'{league}-teams',
    }
response = requests.get('https://www.fotmob.com/api/leagueseasondeepstats',params=params)
data = response.json()

In [4]:
df = pd.DataFrame()

In [5]:
for x in range(1):
    stat = data['statsList'][x]['name']
    params ={
    'id':f'{league_id}',
    'season':f'{season_id}',
    'type':'teams',
    'stat':stat,
    'slug':f'{league}-teams',
    }
    response = requests.get('https://www.fotmob.com/api/leagueseasondeepstats',params=params)
    data = response.json()
    df_temp = pd.DataFrame(data['statsData'])
    for y in range(len(df_temp)):
        df_temp['statValue'][y] = df_temp['statValue'][y]['value']
    df_temp = df_temp.drop(columns=['id','teamId','substatValue','rank','type'])
    stat_name = data['statsList'][x]['title']
    df_temp.rename(columns={'statValue':stat_name},inplace=True)
    df = pd.concat([df,df_temp],axis=1)

In [6]:
df

,name,Accurate crosses per match
0,Huddersfield Town,5.2
1,West Bromwich Albion,5
2,Swansea City,5
3,Ipswich Town,4.8
4,Leicester City,4.6
5,Norwich City,4.6
6,Southampton,4.3
7,Millwall,4.2
8,Coventry City,4.2
9,Stoke City,4.2


In [7]:
for x in range(1,len(data['statsList'])):
    stat = data['statsList'][x]['name']
    params ={
    'id':f'{league_id}',
    'season':f'{season_id}',
    'type':'teams',
    'stat':stat,
    'slug':f'{league}-teams',
    }
    response = requests.get('https://www.fotmob.com/api/leagueseasondeepstats',params=params)
    data = response.json()
    df_temp = pd.DataFrame(data['statsData'])
    for y in range(len(df_temp)):
        df_temp['statValue'][y] = df_temp['statValue'][y]['value']
    df_temp = df_temp.drop(columns=['id','teamId','substatValue','rank','type'])
    stat_name = data['statsList'][x]['title']
    df_temp.rename(columns={'statValue':stat_name},inplace=True)
    df = pd.merge(df,df_temp,how='outer',on='name')
df.fillna(0,inplace=True)

In [8]:
for x in df.columns[1:]:
    df[x] = df[x].astype(float)

In [9]:
#change depending on league
for x in df.columns[1:]:
    if 'per match' in x:
        df[x] = df[x]
    elif 'Average' in x:
        df[x] = df[x]
    else: 
        df[x] = (df[x]/no_games).round(1)

In [10]:
df.columns

Index(['name', 'Accurate crosses per match', 'Accurate long balls per match',
       'Accurate passes per match', 'Average possession', 'Big chances',
       'Big chances missed', 'Clean sheets', 'Clearances per match', 'Corners',
       'Expected goals', 'FotMob rating', 'Fouls per match',
       'Goals conceded per match', 'Goals per match',
       'Interceptions per match', 'Penalties awarded', 'Penalties conceded',
       'Possession won final 3rd per match', 'Red cards', 'Saves per match',
       'Shots on target per match', 'Successful tackles per match',
       'Touches in opposition box', 'xG conceded', 'Yellow cards'],
      dtype='object')

In [11]:
df.to_excel(f"C:\\Users\\George\\Documents\\Football Analytics\\Data\\{league} fotmob.xlsx",index=False)